In [1]:
!pip install kaggle

In [2]:
import os

os.environ['KAGGLE_USERNAME'] = "pallaviingole"
os.environ['KAGGLE_KEY'] = "aa4c8c38b6fc3563c00f86ee1900363e"

In [3]:
!kaggle datasets download -d hhs/health-insurance-marketplace --path zip_data/

 99% 823M/829M [00:06<00:00, 151MB/s]
100% 829M/829M [00:06<00:00, 129MB/s]


In [4]:
!unzip -o zip_data/health-insurance-marketplace.zip -d raw_data/

Archive:  zip_data/health-insurance-marketplace.zip
  inflating: raw_data/BenefitsCostSharing.csv  
  inflating: raw_data/BusinessRules.csv  
  inflating: raw_data/Crosswalk2015.csv  
  inflating: raw_data/Crosswalk2016.csv  
  inflating: raw_data/Network.csv    
  inflating: raw_data/PlanAttributes.csv  
  inflating: raw_data/Rate.csv       
  inflating: raw_data/ServiceArea.csv  
  inflating: raw_data/database.sqlite  
  inflating: raw_data/hashes.txt     
  inflating: raw_data/raw/._.sentinel  
  inflating: raw_data/raw/._2014     
  inflating: raw_data/raw/._2015     
  inflating: raw_data/raw/._2016     
  inflating: raw_data/raw/._Benefits_Cost_Sharing_PUF.csv  
  inflating: raw_data/raw/._Benefits_Cost_Sharing_PUF_2015-12-08.csv  
  inflating: raw_data/raw/._Business_Rules_PUF.csv  
  inflating: raw_data/raw/._Business_Rules_PUF_2015-12-08.csv  
  inflating: raw_data/raw/._Business_Rules_PUF_Reformat.csv  
  inflating: raw_data/raw/._Machine_Readable_PUF_2015-12-21.xlsx  
  infl

In [5]:
import pandas as pd

In [7]:
rate = pd.read_csv("/content/raw_data/Rate.csv")
rate = rate[["BusinessYear", "PlanId", "Age", "IndividualRate"]].copy()

rate2 = rate[rate['Age'] != 'Family Option'].copy()

cond1 = (rate2['Age'] == '0-20')
cond2 = (rate2['Age'] != '0-20')

rate2['AgeCategory'] = ''

rate2.loc[cond1, 'AgeCategory'] = 'Child'
rate2.loc[cond2, 'AgeCategory'] = 'Adult'

rate_pivot = rate2.groupby(['BusinessYear', 'PlanId', 'AgeCategory'])['IndividualRate'].mean().round(2).reset_index(name='AverageRate')

rate_pivot['AgeCategory'].fillna('Both', inplace=True)
rate_pivot['AverageRate'].fillna(0, inplace=True)

rate_pivot


,BusinessYear,PlanId,AgeCategory,AverageRate
0,2014,10191NJ0030001,Adult,473.89
1,2014,10191NJ0030001,Child,180.18
2,2014,10191NJ0030002,Adult,526.16
3,2014,10191NJ0030002,Child,200.05
4,2014,10191NJ0040001,Adult,180.18
...,...,...,...,...
46922,2016,99969OH0080008,Child,145.82
46923,2016,99969OH0080009,Adult,473.34
46924,2016,99969OH0080009,Child,179.97
46925,2016,99969OH0080010,Adult,473.34


In [8]:
rate_pivot.isna().sum()

BusinessYear    0
PlanId          0
AgeCategory     0
AverageRate     0
dtype: int64

In [9]:
import re

def extract_age_group(text):
    child_match = re.search(r'Child', text, re.IGNORECASE)
    adult_match = re.search(r'Adult', text, re.IGNORECASE)

    if child_match:
        return 'Child'
    elif adult_match:
        return 'Adult'
    else:
        return 'Both'

benefits = pd.read_csv('/content/raw_data/BenefitsCostSharing.csv')[["BenefitName", "BusinessYear", "PlanId", "StateCode"]]

benefits['PlanId'] = benefits['PlanId'].str.slice(0, -3)

benefits["AgeCategory"] = benefits["BenefitName"].apply(extract_age_group)
benefits = benefits.dropna()

merged_data = pd.merge(benefits, rate_pivot, on=['BusinessYear', 'PlanId', 'AgeCategory'], how='left')
merged_data = merged_data.rename(columns={"IndividualRate": "AverageRate"})
merged_data['AverageRate'].fillna(0, inplace=True)

merged_data

<ipython-input-9-4b1c27dfe6f6>:14: DtypeWarning: Columns (3,6,9,10,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  benefits = pd.read_csv('/content/raw_data/BenefitsCostSharing.csv')[["BenefitName", "BusinessYear", "PlanId", "StateCode"]]


,BenefitName,BusinessYear,PlanId,StateCode,AgeCategory,AverageRate
0,Routine Dental Services (Adult),2014,21989AK0010001,AK,Adult,32.00
1,Dental Check-Up for Children,2014,21989AK0010001,AK,Child,29.00
2,Basic Dental Care - Child,2014,21989AK0010001,AK,Child,29.00
3,Orthodontia - Child,2014,21989AK0010001,AK,Child,29.00
4,Major Dental Care - Child,2014,21989AK0010001,AK,Child,29.00
...,...,...,...,...,...,...
5048403,Major Dental Care - Child,2016,96480WV0110003,WV,Child,19.14
5048404,Basic Dental Care - Adult,2016,96480WV0110003,WV,Adult,99.99
5048405,Orthodontia - Adult,2016,96480WV0110003,WV,Adult,99.99
5048406,Major Dental Care - Adult,2016,96480WV0110003,WV,Adult,99.99


In [ ]:
merged_data.isna().sum()

BenefitName     0
BusinessYear    0
PlanId          0
StateCode       0
AgeCategory     0
AverageRate     0
dtype: int64

In [10]:
!mkdir transformed

In [12]:
output_directory = '/content/transformed_data'
os.makedirs(output_directory, exist_ok=True)


In [13]:
merged_data.to_csv(os.path.join(output_directory, 'transformed.csv'), index=False)

In [14]:
# Read the saved CSV file
output_file_path = '/content/transformed_data/transformed.csv'
output_data = pd.read_csv(output_file_path)

# Display the first few rows of the output data
print(output_data.head())


                       BenefitName  BusinessYear          PlanId StateCode  \
0  Routine Dental Services (Adult)          2014  21989AK0010001        AK   
1     Dental Check-Up for Children          2014  21989AK0010001        AK   
2        Basic Dental Care - Child          2014  21989AK0010001        AK   
3              Orthodontia - Child          2014  21989AK0010001        AK   
4        Major Dental Care - Child          2014  21989AK0010001        AK   

  AgeCategory  AverageRate  
0       Adult         32.0  
1       Child         29.0  
2       Child         29.0  
3       Child         29.0  
4       Child         29.0  


In [22]:
import re
import unittest

def extract_age_group(text):
    child_match = re.search(r'Child', text, re.IGNORECASE)
    adult_match = re.search(r'Adult', text, re.IGNORECASE)

    if child_match:
        return 'Child'
    elif adult_match:
        return 'Adult'
    else:
        return 'Both'

class TestExtractAgeGroupFunction(unittest.TestCase):
    def test_child_match(self):
        result = extract_age_group("Child Health Benefit")
        self.assertEqual(result, 'Child')

    def test_adult_match(self):
        result = extract_age_group("Adult Health Benefit")
        self.assertEqual(result, 'Adult')

    def test_both_match(self):
        result = extract_age_group("Health Benefit")
        self.assertEqual(result, 'Both')

    def test_case_insensitivity(self):
        result = extract_age_group("child health benefit")
        self.assertEqual(result, 'Child')

        result = extract_age_group("ADULT health benefit")
        self.assertEqual(result, 'Adult')

    def test_no_match(self):
        result = extract_age_group("Some Other Benefit")
        self.assertEqual(result, 'Both')

# To run the tests in the notebook
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestExtractAgeGroupFunction))


.....
----------------------------------------------------------------------
Ran 5 tests in 0.013s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>